# Data and import

## Initial imports

/!\ **fredpy python api is needed to fork data, requires FRED_API_KEY="your_api_key" in file .zshrc and .bashrc** /!\

- Making a request for a key : https://fred.stlouisfed.org/docs/api/api_key.html
- Possibly add FRED_API_KEY="your_api_key" in files .zshrc and .bashrc of computer if code cannot run
- Otherwise try (,api_key="my_api_key") in .get_series of function fred_data
- **Otherwise the data is saved in "data_save.csv"**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
import pyfredapi as pf
import MacroRandomForest as MRF
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import itertools
from math import *

# Use matplotlib's 'classic' style, set figure facecolor to white
plt.style.use('classic')
plt.rcParams.update({'figure.facecolor': 'white'})
matplotlib.rcParams['figure.figsize'] = [20, 9]

global start_date, end_date, window_dates
start_date = np.datetime64('1980-01-01')
end_date = np.datetime64('2023-12-01')


start_covid = np.datetime64('2020-01-01')
end_covid = np.datetime64("2021-04-01")

start_gfc = np.datetime64("2007-05-01")
end_gfc = np.datetime64("2009-02-01")


current_date = start_date
window_dates = []

while current_date <= end_date:
    window_dates.append(current_date)
    current_date = np.datetime64(current_date, 'M') + np.timedelta64(1, 'M')

window_dates = np.array(window_dates, dtype='datetime64[M]').tolist()

gdpm = pd.read_excel("US-Monthly-GDP-History-Data.xlsx",sheet_name="Data")
gdpm.drop(columns=['Unnamed: 0'],inplace=True)
gdpm.set_index('Date',inplace=True)

In [ ]:
df_supp = pd.read_csv("fred_data_new_stat.csv")
df_supp.rename(columns={"Unnamed: 0":"dates"},inplace=True)
df_supp.set_index("dates",drop=True,inplace=True)
df_supp.index.name= None
df_supp.index = pd.to_datetime(df_supp.index)
display(df_supp)

In [ ]:
# Requested series
REQUEST_LIST = ["CSUSHPISA","FEDFUNDS","DGS10","MORTGAGE30US","REAINTRATREARAT10Y","PSAVERT",
                  "CURRCIR","M2SL","INDPRO","UNRATE","CIVPART","CPIAUCSL","MICH",
                  "PCE","DSPIC96","TTLHHM156N","TOTALSL","DPSACBW027SBOG","MSACSR"]

def fred_data(L:list):
    x = pd.DataFrame(columns=L, index=window_dates)
    
    with tqdm(total=len(L), ascii=True) as pbar:
        for var in L:
            try:
                y = pf.get_series(series_id=var,observation_start=str(start_date),observation_end=str(end_date),frequency="m")
                
                # ===== EDIT HERE IN CASE NEEDED - see comment on fred api at the beginning - edit "my_api_key" with FRED provided api key
                #y = pf.get_series(api_key="my_api_key",series_id=var,observation_start=str(start_date),observation_end=str(end_date),frequency="m")
                # ===========================================================================
                
                y.set_index('date',inplace=True)
                y.drop(columns=["realtime_start","realtime_end"],inplace=True)
                x[var] = y.value
            except:
                None
            pbar.update()
    x['SPREAD'] = x['DGS10'] - x['FEDFUNDS']
    x = x.drop(columns=["FEDFUNDS","DGS10"])
    return x

#TODO: data request
data = fred_data(L=REQUEST_LIST)
data[['GDPn','GDPr']] = gdpm[['GDPn','GDPr']] # Monthly GDP data import

# Indicatrices (was not useful)
data['GFC'] = data.index
data['GFC'] = data['GFC'].apply(lambda x: start_gfc<= x <= end_gfc).apply(lambda x: x*1)
data['Covid'] = data.index
data['Covid'] = data['Covid'].apply(lambda x: start_covid<= x <= end_covid).apply(lambda x: x*1)

data = data.dropna()

display(data)
#data.to_csv("Data_save.csv")

## Dataset creation & transformation

In [ ]:
# --- 
#TODO: usual transformation for stationarity 
#? "diff_ln" : 100*(log(x_'t') - log(x_'t-12')) = YoY growth rate
#? "diff" : 100*(x_'t' - x_'t-12') = YoY change in ... rate
#? rolling : special case of "MSACSR" : at time t it is the YoY% growth rate between : the number of houses added between (t) and (t)-11 (= over the past year) and the same number but between (t-12) and (t-12)-11 
#? "level" : no change on data

DATA_TRANSFORM = {
"diff_ln":[
"CSUSHPISA",
"CURRCIR",
"M2SL",
"CPIAUCSL",
"INDPRO",
"PCE",
"DSPIC96",
"TTLHHM156N",
"TOTALSL",
"DPSACBW027SBOG",
"GDPr"
],

"diff":[
"MORTGAGE30US",
"REAINTRATREARAT10Y",
"UNRATE",
"CIVPART",
"MICH"
],

"level":[
"SPREAD",
"PSAVERT"
],

"rolling":["MSACSR"]
}

def log_transform(cell_value):
    try:
        return np.log(float(cell_value))
    except (ValueError, TypeError):
        return cell_value

def data_treat(df,treatment):
        
    x = pd.DataFrame(index=df.index)
    for key in treatment:
        if key=="diff_ln":
            x[treatment[key]] = 100*df[treatment[key]].applymap(log_transform).diff(periods=12)
        elif key=="diff":
            x[treatment[key]] = df[treatment[key]].diff(periods=1)
        elif key=="level":
            x[treatment[key]] = df[treatment[key]]
        else:
            x[treatment[key]] = 100*df[treatment[key]].rolling(12).sum().pct_change(periods=12)
    x = x.dropna()
    return x

df_data = data_treat(df=data,treatment=DATA_TRANSFORM)
VARIABLES_LIST = list(df_data.columns)[1::]
#df_data = df_data - df_data.mean()
df_data

In [ ]:
# ---------------------------------- #

#* Steps ahead forecast
for k in [1,3,6,12]:
    df_data[f"CSUSHPISA+{k}"] = df_data["CSUSHPISA"].shift(periods=-k)

#* > lags [1:3] of all variables
for k in range(1,len(VARIABLES_LIST)):
    if VARIABLES_LIST[k] in ['GFC','Covid']:
        pass
    else:
        for i in range(1,4):
            df_data[f"{VARIABLES_LIST[k]}-{i}"] = df_data[f"{VARIABLES_LIST[k]}"].shift(periods=i)

#* > Lags of dependent variable : up to ...
for k in range(1,4):
    df_data[f"CSUSHPISA-{k}"] = df_data["CSUSHPISA"].shift(periods=k)
    
df_data = df_data.dropna()
df_data = pd.concat([df_data,df_supp],axis=1,join="inner")
df_data

# MRF

## Variable selection

In [ ]:
RESULTATS = {"CSUSHPISA+1":pd.DataFrame(index=df_data.index),"CSUSHPISA+3":pd.DataFrame(index=df_data.index),"CSUSHPISA+6":pd.DataFrame(index=df_data.index),"CSUSHPISA+12":pd.DataFrame(index=df_data.index)}

In [ ]:
# ================================================================================================ #
#! Based on experience when changing any Y_VAR / X_VAR it is better to restart the kernel an re-run all the code until here to avoid any wrongdoing of the following training bloc - save comments later on

#* -----------------------
###* Dependent Variable

#? (1) One-step ahead
#Y_VAR = "CSUSHPISA+1"  

#? (2) 3-step ahead
#Y_VAR = "CSUSHPISA+3"

#? (3) 6-step ahead
Y_VAR = "CSUSHPISA+6"


#* -----------------------
###* Linear part
#? (1) Baseline
#X_VAR = ["CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]

#? (2) Baseline AR
X_VAR = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]

#? (2) 2-Lag AR
#X_VAR = ["CSUSHPISA","CSUSHPISA-1","CSUSHPISA-2"]


#* -----------------------
###* Exogenous Variables

#? (1) S_t
#S_VAR = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]

#? (2) S_t[0-2]
#svari = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]
#S_VAR = svari + [f"{var}-{i}" for var in svari for i in range(1,3)]

#? (3)
S_VAR = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"] + list(df_supp.columns)


# ================================== #
# ================================== #
# Data for the MRF training
data_MRF = pd.DataFrame(index = df_data.index)
data_MRF[[Y_VAR]] = df_data[[Y_VAR]].copy()
data_MRF[X_VAR] = df_data[X_VAR].copy()
data_MRF[S_VAR] = df_data[S_VAR].copy()
data_MRF.T.drop_duplicates().T          #drop duplicate columns if any
Y_pos = data_MRF.columns.get_loc(Y_VAR)
S_pos = [data_MRF.columns.get_loc(s) for s in S_VAR]
X_pos = [data_MRF.columns.get_loc(x) for x in X_VAR]
# ================================== #
# ================================== #


# ======= OOS-WINDOW ============== #
K_OOS = 48
oos_pos = np.arange(len(data_MRF) - K_OOS, len(data_MRF))

# ======= RESULTS ================ #
RESULTATS[Y_VAR]['actual'] = np.nan
RESULTATS[Y_VAR]['actual'].iloc[oos_pos] = df_data[Y_VAR].iloc[oos_pos]

# ================================================================================================ #
display(data_MRF.head())

In [ ]:
data_MRF.iloc[oos_pos]

## Initial comments on some code errors

**If the oos-forecast show too extreme variations / really high spikes and trough for a few months / flat lines : re-run whole code by restarting kernel**

As the process is highly ressource-intensive it appears that when the computer struggles in the forecast phase (especially when the notebook had already been ran) - points are missing and generate abnomrmal spikes/flat lines.
See below for example:

This generates those type of odd results, not due to model miss specification or anything but just due to how the MRF package seem to handle computer struggle.
Reseting the python kernel to run another model seem to solve this problem in most cases.

<div>
<img src="computer_struggle2.png" width="500"/>
</div>

When we look at the predicted coefficients from the MRF some forecasted points are indeed missing, causing the flat lines some are missing for successive dates or a trough when it is a single date missing... 
Only solution is to re-run the code

<div>
<img src="computer_struggle.png" width="500"/>
</div>

For reference this is the correct output (without computer struggle induced missing values).

<div>
<img src="computer_struggle3.png" width="500"/>
</div>

## Model training

In [ ]:
# ================================== #
#TODO: /// MRF FIT ///
MRF_model = MRF.MacroRandomForest(data = data_MRF.copy(),
            y_pos = Y_pos,
            x_pos = X_pos,
            S_pos = S_pos,
            mtry_frac=0.75,
            B = 1000,
            parallelise = True,
            n_cores = -1,
            resampling_opt = 2,
            oos_pos = oos_pos,
            trend_push = 4,
            quantile_rate=None,
            ridge_lambda=0.001,
            subsampling_rate=0.65,
            rw_regul=0.75,
            print_b = True,
            fast_rw = True)

# ================================== #
#TODO: /// MRF training ///
MRF_output = MRF_model._ensemble_loop()

In [ ]:
# Display time-varying parameters
MRF_model.band_plots()

In [ ]:
# Compute results
beta_res = pd.DataFrame(MRF_output['betas'],index=data_MRF.index,columns=['const']+X_VAR)
fit_res = pd.DataFrame(index=beta_res.index)
for x in X_VAR:
    fit_res[x] = beta_res[x]*df_data[x]
fit_res['const'] = beta_res['const']

fit_res['training'] = fit_res.sum(axis=1)
fit_res['oos'] = fit_res['training']
fit_res['training'].iloc[oos_pos] = np.nan
fit_res['oos'].iloc[:len(fit_res)-len(oos_pos)] = np.nan
fit_res['actual'] = df_data[Y_VAR]
fit_res

RESULTATS[Y_VAR]['OOS'] = fit_res['oos']

In [ ]:
fit_res.tail(
)

In [ ]:
display(fit_res[['training','oos','actual']].plot(color=['blue','red','black']))

In [ ]:
print(r2_score(y_true=RESULTATS[Y_VAR].iloc[oos_pos]["actual"],y_pred=RESULTATS[Y_VAR].iloc[oos_pos]["OOS"]))
print(sqrt(mean_squared_error(y_true=RESULTATS[Y_VAR].iloc[oos_pos]["actual"],y_pred=RESULTATS[Y_VAR].iloc[oos_pos]["OOS"])))

In [ ]:
fit_res[["oos","actual"]].iloc[oos_pos].plot()

# Plain RF

In [ ]:
Y_VAR = "CSUSHPISA+3"
#Y_VAR = "CSUSHPISA+6"

In [ ]:
def prepareDataForClassificationRF(dataset,x_reg,y_reg,oos=48):
    """
    generates categorical output column, which is then used
    to create the train and test data
    """ 
    dataset_rf = dataset.copy().reset_index()
    X = (dataset_rf[x_reg])
    y = (dataset_rf[y_reg])
    
    X_train = X.iloc[0:len(X) - oos]
    y_train = y.iloc[0:len(y) - oos]
    
    X_test = X.iloc[len(X) - oos::]
    y_test = y.iloc[len(y) - oos::]
    
    train_index, test_index = dataset.index[0:len(X) - oos], dataset.index[len(X) - oos::]
    
    return X_train, y_train, X_test, y_test, train_index, test_index, dataset

#rfbase = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"] # [X,HP]_t 
rfbase = ["CSUSHPISA"]
xvar_rf = rfbase + [f"{var}-{i}" for var in rfbase for i in range(1,3)]
#xvar_rf = ["CSUSHPISA","CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]
#xvar_rf = ["CPIAUCSL","DSPIC96","SPREAD","MORTGAGE30US","UNRATE"]

X_train, y_train, X_test, y_test, train_index, test_index, dataset  = prepareDataForClassificationRF(dataset=df_data,x_reg=xvar_rf,y_reg=Y_VAR)

In [ ]:
RF_Model = RandomForestRegressor(n_estimators=1000,
                                 max_features=0.60, oob_score=False,
                                 min_samples_split=20)
labels = y_train
features = X_train
rgr=RF_Model.fit(features, labels)
X_test_predict=pd.DataFrame(rgr.predict(X_test), index=test_index)
X_train_predict=pd.DataFrame(rgr.predict(X_train), index=train_index)
RF_predict = X_train_predict.append(X_test_predict).rename(columns={0:'plain_RF_predict'})
RF_predict['actual'] = df_data[Y_VAR].copy()
print("R2_score : ",r2_score(y_true=RF_predict.iloc[oos_pos]['actual'],y_pred=RF_predict.iloc[oos_pos]['plain_RF_predict']))

RF_predict_result = pd.DataFrame(index=RF_predict.index)
RF_predict_result[["training","oos","actual"]] = ""
RF_predict_result["training"].iloc[:len(RF_predict_result)-48] = RF_predict["plain_RF_predict"].iloc[:len(RF_predict_result)-48]
RF_predict_result["training"].iloc[len(RF_predict_result)-48::] = np.nan
RF_predict_result["oos"].iloc[len(RF_predict_result)-48::] = RF_predict["plain_RF_predict"].iloc[len(RF_predict_result)-48::]
RF_predict_result["oos"].iloc[:len(RF_predict_result)-48] = np.nan
RF_predict_result["actual"] = RF_predict["actual"]
RF_predict_result.plot(color=['blue','red','black'])